In [52]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
from urllib import request
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

In [3]:
def html_loader(url):
    request_site = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    webpage = urlopen(request_site).read()
    soup = BeautifulSoup(webpage,"html.parser")
    return(soup)

In [4]:
def cfDecodeEmail(encodedString):
    r = int(encodedString[:2],16)
    email = ''.join([chr(int(encodedString[i:i+2], 16) ^ r) for i in range(2, len(encodedString), 2)])
    return email

In [5]:
name = []
party = []
address = []
phone = []
email = []
region = []
urls = []
urls2 = []

In [6]:
# Url from Scott Parliament with a list of the MSPs
url = "https://www.parliament.scot/msps/current-and-previous-msps"
    
# Create the Soup object from the html
soup = html_loader(url)
    
# Retrieve Name, Party, Region, Email and Url
primary_soup = soup.find_all('div',{"class" : "content-block__content", "class" : "content-block__header"})
secondary_soup = soup.find_all('div',{"class" : "content-block__content", "class" : "content-block__body"})
region_soup = soup.find_all('div',{"class" : "content-block__content", "class" : "content-block__body"})
email_soup = soup.find_all('div',{"class" : "content-block__contact","class" : "new-button new-button--round"})
url_soup = soup.find_all('div',{"class" : "content-block__content", "class" : "content-block__header"})
url3 = 'https://parliament.scot'
    
# Append Names in name list
for row1 in primary_soup:
    name.append(row1.text.strip())
    
print('Names completed')
        
# Append Parties in party list    
for row2 in secondary_soup:
    party.append(row2.p.text.strip())
    
print('Parties completed')
        
# Append Regions in region list    
for row3 in region_soup:
    region.append(row3.p.next_sibling.next_sibling.text.strip())
    
print('Regions completed')

# Manually Extract Address - It is always the same
for row7 in range(0,128):
    address.append('The Scottish Parliament Edinburgh EH99 1SP')
    
print('Addresses completed')

# Retrieve Hrefs for each MP 
for row5 in url_soup:
    urls = row5.a.get('href')
    
        
    # For each Href retrieved create a new Url and convert it into a soup object using html_loader
    # to extract Address
    sophie = (url3 + urls)
    soup = html_loader(sophie)
       
    # Retrieve Email
    email_soup = soup.find_all('div',attrs={"class" : "msp-list__block","class" : "iconBlock"},limit = 1)
    #couldn't find the phone tag
    #phone_soup = soup.find_all('div', text = ' Phone: ')
        
    for row6 in email_soup:
        email.append(cfDecodeEmail(row6.span.attrs['data-cfemail']))
        print('Obtaining Emails')
        

print('Emails completed')

        
        

Brian Adam
George Adam
Karen Adam
Clare Adamson
Bashir Ahmad
Bill Aitken
Wendy Alexander
Alasdair Allan
Christian Allard
Andrew Arbuckle
Tom Arthur
Jackie Baillie
Shiona Baird
Claire Baker
Richard Baker
Jeremy Balfour
Chris Ballance
Michelle Ballantyne
Mark Ballard
Scott Barrie
Jayne Baxter
Claudia Beamish
Colin Beattie
Marco Biagi
Neil Bibby
Bill Bowman
Sarah Boyack
Rhona Brankin
Lesley Brennan
Miles Briggs
Ted Brocklebank
Chic Brodie
Gavin Brown
Keith Brown
Robert Brown
Siobhian Brown
Derek Brownlee
Cameron Buchanan
Ariane Burgess
Margaret Burgess
Alexander Burnett
Bill Butler
Rosemary Byrne
Stephanie Callaghan
Donald Cameron
Aileen Campbell
Colin Campbell
Roderick Campbell
Dennis Canavan
Jackson Carlaw
Finlay Carson
Maggie Chapman
Peter Chapman
Malcolm Chisholm
Foysol Choudhury
Katy Clark
Willie Coffey
Alex Cole-Hamilton
Angela Constance
Maurice Corry
Cathie Craigie
Bruce Crawford
Roseanna Cunningham
Frances Curran
Margaret Curran
David Davidson
Ruth Davidson
Susan Deacon
Donald Dew

In [56]:
#Create a DateFrame with Pandas
df = pd.DataFrame([name, party, region, address])
df = df.transpose()
df.columns = ['Name', 'Party', 'Region', 'Address']
df = df[df["Region"].str.contains("Former") == False]
df = df.reset_index()
df['Email'] = np.array(email)
df = df.drop(['index'], axis=1)
df1 = df[['Name', 'Party', 'Region','Email','Address']]

#Sort by Party and export to Excel
df1.sort_values(by='Party',ascending=True,inplace=True)
df1.to_excel("SCOTGOV.xlsx",sheet_name='SCOTGOV') 
df1




,Name,Party,Region,Email,Address
62,Alison Johnstone,No Party Affiliation,MSP for Lothian (Region),Alison.Johnstone.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
64,Stephen Kerr,Scottish Conservative and Unionist Party,MSP for Central Scotland (Region),Stephen.Kerr.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
39,Murdo Fraser,Scottish Conservative and Unionist Party,MSP for Mid Scotland and Fife (Region),Murdo.Fraser.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
40,Meghan Gallacher,Scottish Conservative and Unionist Party,MSP for Central Scotland (Region),Meghan.Gallacher.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
106,Liz Smith,Scottish Conservative and Unionist Party,MSP for Mid Scotland and Fife (Region),Elizabeth.Smith.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
...,...,...,...,...,...
48,Neil Gray,Scottish National Party,MSP for Airdrie and Shotts (Constituency),Neil.Gray.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
46,Christine Grahame,Scottish National Party,"MSP for Midlothian South, Tweeddale and Lauder...",Christine.Grahame.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
45,Mairi Gougeon,Scottish National Party,MSP for Angus North and Mearns (Constituency),Mairi.Gougeon.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
68,Richard Lochhead,Scottish National Party,MSP for Moray (Constituency),Richard.Lochhead.msp@parliament.scot,The Scottish Parliament Edinburgh EH99 1SP
